In [2]:
import pandas as pd

In [6]:
import sys
import csv

maxInt = sys.maxsize
while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

word_emb=pd.read_csv("glove.6B.100d.txt",delimiter=' ',header=None,engine='python',error_bad_lines=False,quoting=csv.QUOTE_NONE)

In [7]:
word_to_idx={}

words=word_emb[0]
for i in range(0,len(words)):
    word_to_idx[words[i]]=i

In [9]:
conversation=pd.read_csv("Major.txt",delimiter='\+\+\+\$\+\+\+',header=None,engine='python')

In [10]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
indices=[]
keys=set(word_to_idx.keys())
for sent1 in conversation[0]:
    words=word_tokenize(sent1)
    k=[]
    for i in words:
        if i.lower() in keys:
            k.append(word_to_idx[i.lower()])
        else:
            k.append(0)
    indices.append(k)

In [12]:
out=[]
keys=set(word_to_idx.keys())
for sent1 in conversation[1]:
    words=word_tokenize(sent1)
    k=[]
    for i in words:
        if i.lower() in keys:
            k.append(word_to_idx[i.lower()])
        else:
            k.append(0)
    out.append(k)
print(out[0])


[144, 2, 42, 805, 54, 1152, 466, 18, 16346, 2, 84, 13, 10, 15918, 18, 82, 3]


In [13]:
max_in_length=0
for i in indices:
    max_in_length=max(max_in_length,len(i))

In [14]:
max_out_length=0
for i in out:
    max_out_length=max(max_out_length,len(i))

In [15]:
print(max_out_length)

593


In [16]:
from keras.preprocessing.sequence import pad_sequences
padded_in_sentences = pad_sequences(indices,max_in_length, padding='post')

Using TensorFlow backend.


In [17]:
padded_out_sentences = pad_sequences(out,max_out_length, padding='post')

In [18]:
print(padded_in_sentences.shape,padded_out_sentences.shape)

(137935, 370) (137935, 593)


In [19]:
#word_emb.head()
#padded_sentences=padded_sentences[:][:100]

In [20]:
del word_emb[0]

In [21]:
#t=array()

In [22]:
from numpy import array
inp=[]
for i in range(0,5000):
    inter=[]
    for j in  range(0,max_in_length):
        inter.append(word_emb.loc[padded_in_sentences[i][j],:])
    inp.append(array(inter))


In [23]:
in_sent=array(inp)
print(in_sent.shape)

(5000, 370, 100)


In [24]:
inp=[]
for i in range(100,200):
    inter=[]
    for j in  range(0,max_in_length):
        inter.append(word_emb.loc[padded_in_sentences[i][j],:])
    inp.append(array(inter))

In [25]:
test_int=array(inp)
print(test_int.shape)

(100, 370, 100)


In [26]:
out=[]
for i in range(0,5000):
    inter=[]
    for j in  range(0,max_out_length):
        inter.append(word_emb.loc[padded_out_sentences[i][j],:])
    out.append(array(inter))


In [27]:
out_sent=array(out)
#out_sent = out_sent.reshape(max_out_length,1000,100)

print(out_sent.shape)

(5000, 593, 100)


In [28]:
out=[]
for i in range(100,200):
    inter=[]
    for j in  range(0,max_out_length):
        inter.append(word_emb.loc[padded_out_sentences[i][j],:])
    out.append(array(inter))

In [29]:
test_out=array(out)
#test_out = test_out.reshape(max_out_length,1000,100)
print(test_out.shape)

(100, 593, 100)


In [30]:
in_sent1=in_sent[:,:30,:]
out_sent1=out_sent[:,:20,:]

In [31]:
test_int1=test_int[:,:30,:]
test_out1=test_out[:,:20,:]

In [32]:
from keras.layers import LSTM,RepeatVector,Input,Dense
from keras.models import Sequential,Model

In [33]:
import keras

In [34]:
from keras.optimizers import Adam,RMSprop,SGD,Adagrad
from keras.losses import CategoricalCrossentropy 

In [35]:
model_1=Sequential()
model_1.add(LSTM(256))
model_1.add(RepeatVector(20))
model_1.add(LSTM(256,return_sequences=True))
#model_1.add(Dense(100,activation='relu'))
model_1.add(Dense(100,activation='tanh'))

In [36]:
#opt = SGD(lr=0.01, momentum=0.9, decay=0.01)

In [37]:
model_1.compile(optimizer=RMSprop(learning_rate=0.001),loss=keras.losses.cosine_proximity,metrics=['accuracy'])

In [38]:
x=model_1.fit(in_sent1,out_sent1,batch_size=64,epochs=30)

Epoch 1/30
5000/5000 [==============================] - 36s 7ms/step - loss: -0.3711 - accuracy: 0.3346
Epoch 2/30
5000/5000 [==============================] - 29s 6ms/step - loss: -0.3820 - accuracy: 0.3389
Epoch 3/30
5000/5000 [==============================] - 30s 6ms/step - loss: -0.3829 - accuracy: 0.3389
Epoch 4/30
5000/5000 [==============================] - 31s 6ms/step - loss: -0.3832 - accuracy: 0.3388
Epoch 5/30
5000/5000 [==============================] - 29s 6ms/step - loss: -0.3835 - accuracy: 0.3389
Epoch 6/30
5000/5000 [==============================] - 28s 6ms/step - loss: -0.3836 - accuracy: 0.3389
Epoch 7/30
5000/5000 [==============================] - 30s 6ms/step - loss: -0.3838 - accuracy: 0.3388
Epoch 8/30
5000/5000 [==============================] - 31s 6ms/step - loss: -0.3839 - accuracy: 0.3388
Epoch 9/30
5000/5000 [==============================] - 31s 6ms/step - loss: -0.3840 - accuracy: 0.3388
Epoch 10/30
5000/5000 [==============================] - 29s 6ms

In [40]:
_,val_acc = model_1.evaluate(test_int1,test_out1, batch_size=64)
print(val_acc)

100/100 [==============================] - 0s 2ms/step
0.29249998927116394
